## Feature Exploration & Cleaning: Census Data, Hard to Count Populations
 
Data from https://www.censushardtocountmaps2020.us/

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn_pandas import DataFrameMapper

In [2]:
conn=sqlite3.connect('COVID19_county_data.db')
cursor= conn.cursor()

In [3]:
#Create function to query SQL data
def query_data(sql_statement):
    df=pd.read_sql(sql_statement, conn)
    #cursor.execute(sql_statement)
    return df.to_dict('records')

In [31]:
df=pd.read_excel('pdb2017tract_2010MRR_2018ACS_US.xlsx', skiprows=5) 

In [32]:
df.head()

,GEOIDtxt,StateFIPS,StateAbb,StateName,CountyFIPS,CountyName,TractFIPS,MailReturnRateCen2010,LowResponseScore,PctTotPop,...,PctMoved90to99_TotHHDenom,PctMoved1989earlier_TotHHDenom,PctIntFirstEng_TotHUDenom,PctIntFirstBiling_TotHUDenom,PctIntChoiceEng_TotHUDenom,PctIntChoiceBiling_TotHUDenom,PctMailTypeNotKnown_TotHUDenom,PctUpdateEnumerate_TotHUDenom,PctRemoteAlaska_TotHUDenom,PctUpdateLeave_TotHUDenom
0,1001020100,1,AL,Alabama,1,Autauga County,201.0,83.5,16.1,1,...,0.237908,0.099346,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001020200,1,AL,Alabama,1,Autauga County,202.0,81.3,23.2,1,...,0.098748,0.275382,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1001020300,1,AL,Alabama,1,Autauga County,203.0,79.5,20.7,1,...,0.148920,0.156636,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001020400,1,AL,Alabama,1,Autauga County,204.0,83.5,15.6,1,...,0.094570,0.244661,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001020500,1,AL,Alabama,1,Autauga County,205.0,77.3,17.1,1,...,0.102300,0.031624,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
#Create 2 and 3 digit codes to make FIPS code column
df['StateFIPS'] = df['StateFIPS'].apply(lambda x: str(int(x)).zfill(2))
df['CountyFIPS'] = df['CountyFIPS'].apply(lambda x: str(int(x)).zfill(3))

In [34]:
def create_county_col():
    fips=[]
    records=df[['StateFIPS', 'CountyFIPS']].to_dict('records')
    for record in records:
        merge_fips= f"{record['StateFIPS']}{record['CountyFIPS']}"
        fips.append(merge_fips)
    
    return fips
        

In [35]:
df['FIPS']=create_county_col()

In [36]:
df.head()

,GEOIDtxt,StateFIPS,StateAbb,StateName,CountyFIPS,CountyName,TractFIPS,MailReturnRateCen2010,LowResponseScore,PctTotPop,...,PctMoved1989earlier_TotHHDenom,PctIntFirstEng_TotHUDenom,PctIntFirstBiling_TotHUDenom,PctIntChoiceEng_TotHUDenom,PctIntChoiceBiling_TotHUDenom,PctMailTypeNotKnown_TotHUDenom,PctUpdateEnumerate_TotHUDenom,PctRemoteAlaska_TotHUDenom,PctUpdateLeave_TotHUDenom,FIPS
0,1001020100,01,AL,Alabama,001,Autauga County,201.0,83.5,16.1,1,...,0.099346,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01001
1,1001020200,01,AL,Alabama,001,Autauga County,202.0,81.3,23.2,1,...,0.275382,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,01001
2,1001020300,01,AL,Alabama,001,Autauga County,203.0,79.5,20.7,1,...,0.156636,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01001
3,1001020400,01,AL,Alabama,001,Autauga County,204.0,83.5,15.6,1,...,0.244661,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01001
4,1001020500,01,AL,Alabama,001,Autauga County,205.0,77.3,17.1,1,...,0.031624,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01001


In [63]:
sel_df=df[['FIPS', 'StateAbb', 'StateName','CountyName','MailReturnRateCen2010', 'LowResponseScore', 'PctTotPopBornOutUS_TotPopDenom', 'PctNoInternet_TotHHDenom']].groupby(['FIPS']).mean()

In [64]:
sel_df.head()

,MailReturnRateCen2010,LowResponseScore,PctTotPopBornOutUS_TotPopDenom,PctNoInternet_TotHHDenom
FIPS,,,,
01001,82.741667,19.391667,0.018787,0.218865
01003,15692.084375,15641.006250,0.029703,0.172475
01005,76.122222,24.055556,0.022283,0.347161
01007,71.175000,20.625000,0.011627,0.280822
01009,83.022222,19.344444,0.047752,0.273604


In [65]:
sel_df.isna().sum()

MailReturnRateCen2010             0
LowResponseScore                  0
PctTotPopBornOutUS_TotPopDenom    0
PctNoInternet_TotHHDenom          0
dtype: int64

In [66]:
sel_df.head()

,MailReturnRateCen2010,LowResponseScore,PctTotPopBornOutUS_TotPopDenom,PctNoInternet_TotHHDenom
FIPS,,,,
01001,82.741667,19.391667,0.018787,0.218865
01003,15692.084375,15641.006250,0.029703,0.172475
01005,76.122222,24.055556,0.022283,0.347161
01007,71.175000,20.625000,0.011627,0.280822
01009,83.022222,19.344444,0.047752,0.273604


In [67]:
sel_df.to_sql('hard_to_count', conn, index_label='id', if_exists='replace')

In [68]:
new_df= pd.DataFrame(query_data('SELECT * FROM hard_to_count'))

In [69]:
new_df.head()

,LowResponseScore,MailReturnRateCen2010,PctNoInternet_TotHHDenom,PctTotPopBornOutUS_TotPopDenom,id
0,19.391667,82.741667,0.218865,0.018787,01001
1,15641.006250,15692.084375,0.172475,0.029703,01003
2,24.055556,76.122222,0.347161,0.022283,01005
3,20.625000,71.175000,0.280822,0.011627,01007
4,19.344444,83.022222,0.273604,0.047752,01009


In [72]:
new_df=new_df.rename(columns={'id':'FIPS'})

In [73]:
new_df.head()

,LowResponseScore,MailReturnRateCen2010,PctNoInternet_TotHHDenom,PctTotPopBornOutUS_TotPopDenom,FIPS
0,19.391667,82.741667,0.218865,0.018787,01001
1,15641.006250,15692.084375,0.172475,0.029703,01003
2,24.055556,76.122222,0.347161,0.022283,01005
3,20.625000,71.175000,0.280822,0.011627,01007
4,19.344444,83.022222,0.273604,0.047752,01009


In [75]:
new_df.to_sql('hard_to_count', conn, index_label='id', if_exists='replace')